In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob
import sys
import os

In [2]:
INPUT = os.getcwd()
INPUT = os.path.join(INPUT, "INPUT")
INPUT

'/home/superusuario/Dados/Mapbiomas/newAcuracia/MAPBIOMAS_ACURACIA/Acurracy/CODE/INPUT'

In [3]:

df_array = []

# df = pd.read_csv('INPUT/acc_mapbiomas_col5_v8_2015.csv')
allfiles = glob.glob(os.path.join(INPUT, "*.csv"))
df_from_each_file = (pd.read_csv(f) for f in allfiles)

concat_df   = pd.concat(df_from_each_file, axis=0, ignore_index=True)

print("temos {} filas ".format(concat_df.shape))

temos (2426546, 18) filas 


In [4]:
concat_df['system:index'] = concat_df.index
concat_df.head()

,system:index,AMOSTRAS,BIOMA,CARTA,COUNT,LAT,LON,NEW_PROB,PESO_VOT,POINTEDITE,PROB_AMOS,REINSP,TARGET_FID,VAL_PESO,classification,reference,year,.geo
0,0,NaN,Cerrado,SE-23-Z,2,-2.406225,-43.340607,0.000008,0.000005,False,0.000010,NaN,2526,0.5,4,32,1993,NaN
1,1,NaN,Cerrado,SD-23-X,1,-2.867061,-42.280415,0.000010,0.000010,True,0.000010,NaN,12013,1.0,5,5,1993,NaN
2,2,NaN,Cerrado,SA-23-Z,1,-2.789446,-42.112520,0.000010,0.000014,True,0.000014,NaN,17445,1.0,32,5,1993,NaN
3,3,NaN,Cerrado,SE-24-V,1,-2.819630,-41.905009,0.000009,0.000011,True,0.000011,NaN,5093,1.0,3,5,1993,NaN
4,4,NaN,Mata Atlântica,SC-24-Z,1,-12.904117,-38.677003,0.000008,0.000010,True,0.000010,NaN,6410,1.0,5,5,1993,NaN


In [5]:
concat_df = concat_df[concat_df['AMOSTRAS'] != 'treinamento']
concat_df.shape

(2134401, 18)

In [6]:
 for varbioma in concat_df['BIOMA'].unique():
    dfTemp = concat_df[concat_df['BIOMA'] == varbioma]
    print("bioma {} : filas {}".format(varbioma, dfTemp.shape))

bioma Cerrado : filas (623135, 18)
bioma Mata Atlântica : filas (370290, 18)
bioma Caatinga : filas (272950, 18)
bioma Amazônia : filas (747456, 18)
bioma Pampa : filas (64881, 18)
bioma Pantanal : filas (55689, 18)


In [7]:
def config_class_21(df):	
	agro_filter = (df['classification'] == 21) & (df['reference'].isin([15,19,20]))
	df.loc[agro_filter, 'reference'] = 21
	
	return df

In [8]:
concat_df = config_class_21(concat_df)
concat_df.shape

(2134401, 18)

In [9]:
def config_class_Amazonia(df):
	# print("número pontos Amazonia:  " + str(df.shape[0]))
	agro_filter = (df['BIOMA'] == 'Amazônia') & (df['classification'] == 12) & (df['reference'] == 13)
	df.loc[agro_filter, 'reference'] = 12
	agro_filter = (df['BIOMA'] == 'Amazônia') & (df['classification'] == 4) & (df['reference'] == 13)
	df.loc[agro_filter, 'reference'] = 4		
	
	return df

In [10]:
concat_df = config_class_Amazonia(concat_df)
concat_df.shape

(2134401, 18)

In [11]:
dictRegras = {
    '0':{
        'BIOMA' : 'Amazônia',
        'classification': 12,
        'reference': 13,
        'reference_new': 12
    },
    '1':{
        'BIOMA' : 'Amazônia',
        'classification': 4,
        'reference': 13,
        'reference_new': 4
    },
    '2':{
        'BIOMA' : 'Pantanal',
        'classification': 11,
        'reference': 12,
        'reference_new': 11
    },
    '3':{
        'BIOMA' : 'Cerrado',
        'classification': 13,
        'reference': 12,
        'reference_new': 13
    },
    '4':{
        'BIOMA' : 'Cerrado',
        'classification': 23,
        'reference': 25,
        'reference_new': 23
    },
    '5':{
        'BIOMA' : 'Pampa',
        'classification': 13,
        'reference': 12,
        'reference_new': 13
    },
    '6':{
        'BIOMA' : 'Pampa',
        'classification': 23,
        'reference': 25,
        'reference_new': 23
    },
    '7':{
        'BIOMA' : 'Mata Atlântica',
        'classification': 11,
        'reference': 13,
        'reference_new': 11
    },
}

In [12]:
dictRegras.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7'])

In [14]:
def config_class_biomas(df, indice):
	agro_filter = (df['BIOMA'] == dictRegras[indice]['BIOMA']) & \
                                (df['classification'] == dictRegras[indice]['classification']) \
                                & (df['reference'] == dictRegras[indice]['reference'])
	df.loc[agro_filter, 'reference'] = dictRegras[indice]['reference_new']
	return df

In [16]:
for ind in dictRegras.keys():
    df = config_class_biomas(concat_df, ind)

concat_df.shape

(2134401, 18)

In [17]:
newDF = concat_df[concat_df['BIOMA'] == 'Amazônia']
newDF.head()

,system:index,AMOSTRAS,BIOMA,CARTA,COUNT,LAT,LON,NEW_PROB,PESO_VOT,POINTEDITE,PROB_AMOS,REINSP,TARGET_FID,VAL_PESO,classification,reference,year,.geo
10,10,NaN,Amazônia,SH-22-Z,1,2.095143,-50.402173,0.000013,0.000018,True,0.000018,True,81908,1.0,3,5,1993,NaN
12,12,NaN,Amazônia,SA-23-Z,2,-2.377928,-43.710623,0.000011,0.000007,False,0.000014,NaN,16890,0.5,5,5,1993,NaN
13,13,NaN,Amazônia,SA-23-Z,2,-2.772738,-44.364955,0.000011,0.000007,False,0.000014,NaN,16907,0.5,5,5,1993,NaN
14,14,NaN,Amazônia,SA-23-Z,2,-2.822594,-44.328304,0.000010,0.000007,False,0.000014,NaN,17063,0.5,5,5,1993,NaN
15,15,NaN,Amazônia,SA-23-Z,2,-2.557951,-43.845370,0.000010,0.000007,False,0.000014,NaN,17266,0.5,5,5,1993,NaN


In [20]:
newDF[(newDF['classification'] == 12) & (newDF['reference'] == 13)]

,system:index,AMOSTRAS,BIOMA,CARTA,COUNT,LAT,LON,NEW_PROB,PESO_VOT,POINTEDITE,PROB_AMOS,REINSP,TARGET_FID,VAL_PESO,classification,reference,year,.geo


In [18]:
# newDF[['year', 'reference']].groupby(by= 'reference').count()
pd.crosstab(concat_df.year, concat_df.reference)

reference,3,4,5,9,11,12,13,15,19,20,21,23,24,25,29,30,31,32,33
year,,,,,,,,,,,,,,,,,,,
1985,26943,12763,64,399,519,5209,479,12772,1701,327,1645,43,289,86,91,9,0,1,788
1986,26641,12388,64,400,451,5223,487,13250,1766,331,1677,44,295,91,91,9,0,1,778
1987,26311,12071,64,417,498,5139,473,13785,1823,334,1684,45,296,94,91,10,0,1,767
1988,26085,11770,63,426,517,5060,480,14233,1904,327,1636,45,297,96,91,9,0,1,778
1989,25853,11506,62,439,542,5006,478,14513,1983,321,1678,45,303,95,91,9,0,1,783
1990,25701,11307,62,451,480,5047,484,14742,2043,324,1659,46,309,98,91,10,0,1,779
1991,25575,11142,62,471,515,4994,488,15002,2055,330,1579,45,315,100,91,11,0,1,783
1992,25387,10985,62,485,493,4943,508,15237,2105,338,1603,44,319,102,91,11,0,1,778
1993,25200,10817,62,496,470,4958,490,15439,2153,350,1617,44,328,104,91,11,0,1,773
